In [1]:
from copy import deepcopy
import subprocess
import numpy as np
import open3d as o3d
import trimesh
import os
import time
print(o3d.__version__)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
0.18.0


In [2]:
def subdivide_surface_fitting(decimated_mesh, target_mesh, iterations=1):
    subdivided_mesh = o3d.geometry.TriangleMesh.subdivide_midpoint(decimated_mesh, number_of_iterations=iterations)
    print(subdivided_mesh)
    subdivided_mesh.compute_vertex_normals()
    
    pcd_target = o3d.geometry.PointCloud()
    pcd_target.points = o3d.utility.Vector3dVector(target_mesh.vertices)
    pcd_tree = o3d.geometry.KDTreeFlann(pcd_target)
    subdivided_vertices = np.array(subdivided_mesh.vertices)
    target_vertices = np.array(target_mesh.vertices)
    fitting_vertices = deepcopy(subdivided_vertices)
    
    for i in range(0, len(subdivided_vertices)):
        [k, index, _] = pcd_tree.search_knn_vector_3d(subdivided_vertices[i], 1)
        fitting_vertices[i] = target_vertices[index[0]]

    subdivided_mesh.vertices = o3d.utility.Vector3dVector(fitting_vertices)
    return subdivided_mesh

In [16]:
def read_triangle_mesh_with_trimesh(avatar_name,enable_post_processing=False):
    # EDIT: next 4 lines replace to maintain order even in case of degenerate and non referenced
    # scene_patch = trimesh.load(avatar_name,process=enable_post_processing)
    if enable_post_processing:
        scene_patch = trimesh.load(avatar_name,process=True)
    else:
        scene_patch = trimesh.load(avatar_name,process=False,maintain_order=True) 
    mesh = o3d.geometry.TriangleMesh(
        o3d.utility.Vector3dVector(scene_patch.vertices),
        o3d.utility.Vector3iVector(scene_patch.faces)
    ) 
    if scene_patch.vertex_normals.size:
        mesh.vertex_normals = o3d.utility.Vector3dVector(scene_patch.vertex_normals.copy())
    if scene_patch.visual.defined:
        # either texture or vertex colors if no uvs present.
        if scene_patch.visual.kind == 'vertex':
            mesh.vertex_colors = o3d.utility.Vector3dVector(scene_patch.visual.vertex_colors[:,:3]/255) # no alpha channel support
        elif scene_patch.visual.kind == 'texture':
            uv = scene_patch.visual.uv
            if uv.shape[0] == scene_patch.vertices.shape[0]:
                mesh.triangle_uvs = o3d.utility.Vector2dVector(uv[scene_patch.faces.flatten()])
            elif uv.shape[0] != scene_patch.faces.shape[0] * 3:
                assert False
            else:
                mesh.triangle_uvs = o3d.utility.Vector2dVector(uv)
                if scene_patch.visual.material is not None and scene_patch.visual.material.image is not None:
                    if scene_patch.visual.material.image.mode == 'RGB':
                        mesh.textures = [o3d.geometry.Image(np.asarray(scene_patch.visual.material.image))]
                    else:
                        assert False
        else:
            assert False
    return mesh

In [38]:
GoF = 2
offset = 1
for i in range(0, 9):
    dynamic_deformed = o3d.io.read_triangle_mesh(fr'G:\VS2022Projects\tvm-editing\TVMEditor.Test\bin\Release\net5.0\output\Answering\reference/deformed_reference_mesh_{i+offset:03}.obj')
    original_i = o3d.io.read_triangle_mesh(fr'G:\VS2022Projects\tvm-editing\TVMEditor.Test\bin\Release\net5.0\Data\answering_2000\meshes/mesh_0{i+offset:03}.obj')

    print(dynamic_deformed)
    print(original_i)
    dynamic_deformed.compute_vertex_normals()
    original_i.compute_vertex_normals()
    #o3d.visualization.draw_geometries([reconstruct_dancer_i])
    fitting_mesh_dancer_i = subdivide_surface_fitting(dynamic_deformed, original_i, 1)
    #print(np.array(fitting_mesh_dancer_i.triangles))
    o3d.io.write_triangle_mesh(fr'G:\VS2022Projects\tvm-editing\TVMEditor.Test\bin\Release\net5.0\output\Answering\reference/fitting_mesh_Answering_{i+offset:03}.obj', fitting_mesh_dancer_i, write_vertex_normals=False, write_vertex_colors=False, write_triangle_uvs=False)
    #o3d.visualization.draw_geometries([fitting_mesh_dancer_i])

TriangleMesh with 5819 points and 12000 triangles.
TriangleMesh with 22752 points and 45212 triangles.
TriangleMesh with 23900 points and 48000 triangles.
[Open3D WARNING] Write OBJ can not include triangle normals.
TriangleMesh with 5819 points and 12000 triangles.
TriangleMesh with 22402 points and 44504 triangles.
TriangleMesh with 23900 points and 48000 triangles.
[Open3D WARNING] Write OBJ can not include triangle normals.
TriangleMesh with 5819 points and 12000 triangles.
TriangleMesh with 22632 points and 44982 triangles.
TriangleMesh with 23900 points and 48000 triangles.
[Open3D WARNING] Write OBJ can not include triangle normals.
TriangleMesh with 5819 points and 12000 triangles.
TriangleMesh with 22554 points and 44871 triangles.
TriangleMesh with 23900 points and 48000 triangles.
[Open3D WARNING] Write OBJ can not include triangle normals.
TriangleMesh with 5819 points and 12000 triangles.
TriangleMesh with 22083 points and 43881 triangles.
TriangleMesh with 23900 points an

In [40]:
dataset = "Answering"
loaded_decimated_reference_mesh = o3d.io.read_triangle_mesh(fr'G:\VS2022Projects\tvm-editing\TVMEditor.Test\bin\Release\net5.0\Data\answering_2000\reference_mesh/decimated_reference_mesh.obj', enable_post_processing=False)
print(loaded_decimated_reference_mesh)
print(np.array(loaded_decimated_reference_mesh.triangles))
loaded_decimated_reference_mesh_vertices = np.array(loaded_decimated_reference_mesh.vertices)
loaded_decimated_reference_mesh_vertices

TriangleMesh with 5819 points and 12000 triangles.
[[   0    1    2]
 [   1    3    2]
 [   3    4    2]
 ...
 [5422 5303 5525]
 [5302 5525 5303]
 [3887 1159 1158]]


array([[-0.69275898, -0.93006802, -1.81081009],
       [-0.68782902, -0.87445998, -1.81029999],
       [-0.67611402, -0.76023602, -1.76497006],
       ...,
       [-0.70194799,  0.58867902, -0.26358899],
       [-0.20006099, -0.169802  ,  0.58835799],
       [-0.64339399, -0.39764401,  0.274097  ]])

In [41]:
dataset = "Answering"
loaded_decimated_reference_mesh = o3d.io.read_triangle_mesh(fr'G:\VS2022Projects\tvm-editing\TVMEditor.Test\bin\Release\net5.0\Data\answering_2000\reference_mesh/decimated_reference_mesh.obj', enable_post_processing=False)
print(loaded_decimated_reference_mesh)
loaded_decimated_reference_mesh_vertices = np.array(loaded_decimated_reference_mesh.vertices)
subdivided_decimated_reference_mesh = o3d.geometry.TriangleMesh.subdivide_midpoint(loaded_decimated_reference_mesh, number_of_iterations=1)
#print(subdivided_decimated_reference_mesh)
subdivided_decimated_reference_mesh_vertices = np.array(subdivided_decimated_reference_mesh.vertices)
#o3d.visualization.draw_geometries([subdivided_decimated_reference_mesh])
displacements = []
for i in range(0, 9):
    offset = 1
    fitting_mesh_dancer_i = read_triangle_mesh_with_trimesh(fr'G:\VS2022Projects\tvm-editing\TVMEditor.Test\bin\Release\net5.0\output\{dataset}\reference/fitting_mesh_{dataset}_{i+offset:03}.obj', enable_post_processing=False)
    #fitting_mesh_dancer_i = fitting_mesh_dancer[i]
    print(fitting_mesh_dancer_i, subdivided_decimated_reference_mesh)
    fitting_mesh_vertices = np.array(fitting_mesh_dancer_i.vertices)
    #print(np.array(fitting_mesh_dancer_i.triangles))
    displacement_i = fitting_mesh_vertices - subdivided_decimated_reference_mesh_vertices
    np.savetxt(fr'G:\VS2022Projects\tvm-editing\TVMEditor.Test\bin\Release\net5.0\output\{dataset}\reference/displacements_{dataset}_{i+offset:03}.txt', displacement_i, fmt='%8f')
    displacements.append(displacement_i)



TriangleMesh with 5819 points and 12000 triangles.
TriangleMesh with 23900 points and 48000 triangles. TriangleMesh with 23900 points and 48000 triangles.
TriangleMesh with 23900 points and 48000 triangles. TriangleMesh with 23900 points and 48000 triangles.
TriangleMesh with 23900 points and 48000 triangles. TriangleMesh with 23900 points and 48000 triangles.
TriangleMesh with 23900 points and 48000 triangles. TriangleMesh with 23900 points and 48000 triangles.
TriangleMesh with 23900 points and 48000 triangles. TriangleMesh with 23900 points and 48000 triangles.
TriangleMesh with 23900 points and 48000 triangles. TriangleMesh with 23900 points and 48000 triangles.
TriangleMesh with 23900 points and 48000 triangles. TriangleMesh with 23900 points and 48000 triangles.
TriangleMesh with 23900 points and 48000 triangles. TriangleMesh with 23900 points and 48000 triangles.
TriangleMesh with 23900 points and 48000 triangles. TriangleMesh with 23900 points and 48000 triangles.


In [24]:
for i in range(0, 9):
    offset = 1
    displacement = np.loadtxt(
        fr'G:\VS2022Projects\tvm-editing\TVMEditor.Test\bin\Release\net5.0\output\Answering\reference/displacements_{dataset}_{i + offset:03}.txt')
    pcd = o3d.geometry.PointCloud()
    points = displacement
    pcd.points = o3d.utility.Vector3dVector(points)
    print(pcd)
    #o3d.io.write_point_cloud(r'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Dancer\reference_mesh/dis.ply', pcd)
    points = np.asarray(pcd.points)
    dtype = o3d.core.float32
    p_tensor = o3d.core.Tensor(points, dtype=dtype)
    pc = o3d.t.geometry.PointCloud(p_tensor)
    o3d.t.io.write_point_cloud(
        fr'G:\VS2022Projects\tvm-editing\TVMEditor.Test\bin\Release\net5.0\Data\{dataset}_2000\reference_mesh/dis_{dataset}_{i + offset:03}.ply',
        pc, write_ascii=True)

PointCloud with 23900 points.
PointCloud with 23900 points.
PointCloud with 23900 points.
PointCloud with 23900 points.
PointCloud with 23900 points.
PointCloud with 23900 points.
PointCloud with 23900 points.
PointCloud with 23900 points.
PointCloud with 23900 points.
